In [1]:
## Forward model a data

In [2]:
import SimPEG as simpeg
import simpegMT as simpegmt
from simpegMT.Utils.dataUtils import rec2ndarr
import cPickle as pickle

In [19]:
# Make the mesh.
mTensor = simpeg.Utils.meshTensor
cSize = [50,20]
# Cells constant size mesh
hx = mTensor([(cSize[0],50)])
hy = mTensor([(cSize[0],50)])
hz = mTensor([(cSize[1],48)])
x0 = np.array([-1250,-1250,- 30*20])
mesh3dCons = simpeg.Mesh.TensorMesh([hx,hy,hz],x0)
# With padding
hPad = mTensor([(cSize[0],5,1.5)])
aPad = mTensor([(cSize[1],9,1.5)])
bPad = mTensor([(cSize[1],9,-1.5)])
hxPad = np.hstack((hPad[::-1],mTensor([(cSize[0],40)]),hPad))
hyPad = np.hstack((hPad[::-1],mTensor([(cSize[0],40)]),hPad))
hzPad = np.hstack((bPad,mTensor([(cSize[1],30)]),aPad))
x0Pad = np.array([-(np.sum(hPad)+1000),-(np.sum(hPad)+1000),-(np.sum(bPad)+(20*30))])
mesh3d = simpeg.Mesh.TensorMesh([hxPad,hyPad,hzPad],x0Pad)

In [20]:

# Read the model
modelname = "simpegTDmodel.con"

# Load the model to the uniform cell mesh
modelUniCell = simpeg.Utils.meshutils.readUBCTensorModel(modelname,mesh3dCons)

# Load the model to the mesh with padding cells
modelT = simpeg.Utils.meshutils.readUBCTensorModel(modelname,mesh3d)
# Adjust the model to reflect changes in the mesh (fewer aircells)
modMat = mesh3d.r(modelT,'CC','CC','M')
modNewMat = np.ones((50,50,48))*modMat[0,0,0]
modNewMat[:,:,9::] = modMat[:,:,:-9]
modelTD = mesh3d.r(modNewMat,'CC','CC','V')


In [21]:
# Load the model to the uniform cell mesh
modelUniCell = simpeg.Utils.meshutils.readUBCTensorModel(modelname,mesh3dCons)
# Save as a vtk file
simpeg.Utils.meshutils.writeVTRFile('modelTDuniMesh.vtr',mesh3dCons,{'S/m':modelUniCell})

In [6]:
# Load the model to the mesh with padding cells
# modelTD = simpeg.Utils.meshutils.readUBCTensorModel(modelname,mesh3d)
# Save as a vtk file
simpeg.Utils.meshutils.writeVTRFile('modelTDpaddedMesh.vtr',mesh3d,{'S/m':modelTD})

In [22]:
# Define the data locations
xG,yG = np.meshgrid(np.linspace(-700,700,8),np.linspace(-700,700,8))
zG = np.zeros_like(xG)
locs = np.hstack((simpeg.mkvc(xG.ravel(),2),simpeg.mkvc(yG.ravel(),2),simpeg.mkvc(zG.ravel(),2)))

In [23]:
# Make the receiver list
rxList = []
for rxType in ['zxxr','zxxi','zxyr','zxyi','zyxr','zyxi','zyyr','zyyi']:
    rxList.append(simpegmt.SurveyMT.RxMT(locs,rxType))    
# Source list
srcList =[]
freqs = np.logspace(4,0,17)
for freq in freqs:
    srcList.append(simpegmt.SurveyMT.srcMT_polxy_1Dprimary(rxList,freq))
# Survey MT
survey = simpegmt.SurveyMT.SurveyMT(srcList)


# Forward model the data
if False:
    fields = problem.fields(modelTD)
    mtData = survey.projectFields(fields)

In [9]:
# Load the data
mtSeogiData = simpegmt.DataMT.DataMT(survey,np.load('seogiModel_MTdata.npy'))
mtSeogirecData = mtSeogiData.toRecArray('Complex')

/home/gudni/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:2834: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a record
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  if (obj.__array_interface__["data"][0]
/home/gudni/anaconda/lib/python2.7/site-packages/numpy/ma/core.py:2835: FutureWarning: Numpy has detected that you (may be) writing to an array returned
by numpy.diagonal or by selecting multiple fields in a record
array. This code will likely break in a future numpy release --
see numpy.diagonal or arrays.indexing reference docs for details.
The quick fix is to make an explicit copy (e.g., do
arr.diagonal().copy() or arr[['f0','f1']].copy()).
  != self.__array_interface__["data"][0]):


In [10]:
sys.path.append('/home/gudni/Dropbox/code/python/MTview/')
import interactivePlotFunctions as iPf

In [11]:
% matplotlib qt
# Plot the data
mtSeogiMap = iPf.MTinteractiveMap([mtSeogirecData])

In [13]:
mtSeogirecData.shape

(1088,)

In [30]:
# Add noise to the data
std = 0.05 # 5% std
if os.path.isfile('seogiModel_MTdata.npy') and os.path.isfile('seogiModel_dobs.npy'):
    d_true = np.load('seogiModel_MTdata.npy')
    d_obs = np.load('seogiModel_dobs.npy')
else:
    d_true = simpeg.mkvc(mtSeogiData)
    d_obs = d_true + std*abs(d_true)*np.random.randn(*d_true.shape)
    np.save('seogiModel_dobs.npy',d_obs)
# Assign the dobs
survey.dtrue = d_true
survey.dobs = d_obs
survey.std = np.abs(survey.dobs*std) + 0.01*np.linalg.norm(survey.dobs) #survey.dobs*0 + std
# Assign the data weight
survey.Wd = 1/survey.std #(abs(survey.dobs)*survey.std)

In [31]:
# Make an MTdata object with dobs
mtSeogiDobs = simpegmt.DataMT.DataMT(survey,survey.dobs)

In [32]:
# Run the setup
mt1DdataZyxList = simpegmt.Utils.dataUtils.convert3Dto1Dobject(mtSeogiDobs,'zyx')
mt1DdataZxyList = simpegmt.Utils.dataUtils.convert3Dto1Dobject(mtSeogiDobs,'zxy')

In [33]:
m1d = simpeg.Mesh.TensorMesh([mesh3d.hz],[mesh3d.x0[-1]])
# Setup the problem object
sigma1d = mesh3d.r(modelTD,'CC','CC','M')[0,0,:] # Use the edge column as a background model
# Set the mapping
actMap = simpeg.Maps.ActiveCells(m1d, sigma1d > 1e-7, np.log(1e-8), nC=m1d.nCx)
mappingExpAct = simpeg.Maps.ExpMap(m1d) * actMap
problem = simpegmt.ProblemMT1D.eForm_psField(m1d,sigmaPrimary = sigma1d,mapping=mappingExpAct)
problem.verbose = False



In [49]:
sigma1d

array([  5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   5.00000000e-03,   5.00000000e-03,
         5.00000000e-03,   1.00000000e-03,   1.00000000e-03,
         1.00000000e-03,   1.00000000e-03,   1.00000000e-03,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08])

In [66]:
# Define a function to run the forward problem
def runInversionModel(data,problem,m1d,nameflag):
    problem.unpair()
    data.survey.unpair()
    problem.pair(data.survey)
    # Define a counter
    C =  simpeg.Utils.Counter()
    # Set the optimization
    opt = simpeg.Optimization.InexactGaussNewton(maxIter = 10)
    opt.counter = C
    opt.LSshorten = 0.5
    opt.remember('xc')
    # Data misfit
    dmis = simpeg.DataMisfit.l2_DataMisfit(data.survey)
    # Regularization
    regMesh = simpeg.Mesh.TensorMesh([m1d.hx[problem.mapping.sigmaMap.maps[-1].indActive]],m1d.x0)
    reg = simpeg.Regularization.Tikhonov(regMesh)
    reg.smoothModel = False
    reg.alpha_s = 1e-6
    reg.alpha_x = 1.
    # reg.alpha_xx = .001
    # Inversion problem
    invProb = simpeg.InvProblem.BaseInvProblem(dmis, reg, opt)
    invProb.counter = C
    # Beta cooling
    beta = simpeg.Directives.BetaSchedule()
    betaest = simpeg.Directives.BetaEstimate_ByEig(beta0_ratio=0.75)
    targmis = simpeg.Directives.TargetMisfit()
#     targmis.target = .6 * data.survey.nD
#     print 'Target misfit is {:.0f}'.format(targmis.target)
    locs = np.ones((regMesh.nC,1))*data.survey.srcList[0].rxList[0].locs[:,:-1]
    saveModel = simpeg.Directives.SaveModelEveryIteration()
    saveModel.fileName = 'Inversion_modAt{:.0f}_{:.0f}'.format(locs[0,0],locs[0,1])
    print saveModel.fileName
    # Create an inversion object
    inv = simpeg.Inversion.BaseInversion(invProb, directiveList=[beta,betaest,targmis])
    # Run
    m_0 = np.log(1e-2+0*problem.sigmaPrimary[problem.mapping.sigmaMap.maps[-1].indActive])
    problem.survey.mtrue = m_0
    mopt = inv.run(m_0)
    # Save the model
    
    xyzv = np.hstack((locs,simpeg.mkvc(regMesh.gridCC,2),simpeg.mkvc(mopt,2)))
    np.save('xyzmod_{:s}_{:.0f}_{:.0f}.npy'.format(nameflag,locs[0,0],locs[0,1]),xyzv)
    return mopt
    
    
    

In [67]:
mopt = runInversionModel(dat,problem,m1d,'zyx')


Inversion_modAt-700_-700
SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
                    ***Done using same solver as the problem***
SimPEG.l2_DataMisfit is creating default weightings for Wd.
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
   0  9.42e+01  1.17e+02  0.00e+00  1.17e+02    3.97e+01      0              
------------------------- STOP! -------------------------
1 : |fc-fOld| = 0.0000e+00 <= tolF*(1+|f0|) = 1.1766e+01
0 : |xc-x_last| = 5.8426e+00 <= tolX*(1+|x0|) = 2.9759e+00
0 : |proj(x-g)-x|    = 3.9728e+01 <= tolG          = 1.0000e-01
0 : |proj(x-g)-x|    = 3.9728e+01 <= 1e3*eps       = 1.0000e-02
0 : maxIter   =      10    <= iter          =      1
------------------------- DONE! -------------------------


In [69]:
for dat in mt1DdataZyxList:
    runInversionModel(dat,problem,m1d,'zyx')

for dat in mt1DdataZxyList:
    runInversionModel(dat,problem,m1d,'zxy')

Inversion_modAt-700_-700
SimPEG.InvProblem will set Regularization.mref to m0.
SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
                    ***Done using same solver as the problem***
SimPEG.l2_DataMisfit is creating default weightings for Wd.
============================ Inexact Gauss Newton ============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
   0  1.43e+02  1.17e+02  0.00e+00  1.17e+02    3.97e+01      0              
------------------------- STOP! -------------------------
1 : |fc-fOld| = 0.0000e+00 <= tolF*(1+|f0|) = 1.1766e+01
0 : |xc-x_last| = 5.8497e+00 <= tolX*(1+|x0|) = 2.9759e+00
0 : |proj(x-g)-x|    = 3.9728e+01 <= tolG          = 1.0000e-01
0 : |proj(x-g)-x|    = 3.9728e+01 <= 1e3*eps       = 1.0000e-02
0 : maxIter   =      10    <= iter          =      1
------------------------- DONE! -------------------------
I

In [24]:
seogizxy1Dmods = np.vstack(([np.load(f) for f in glob('xyzmod_zxy*.npy')]))
seogizyx1Dmods = np.vstack(([np.load(f) for f in glob('xyzmod_zyx*.npy')]))

In [29]:
airInd = modelTD ==1e-8
# Interpolate the 1D models
from scipy.interpolate import griddata
mod3d_zxy1dMod = griddata(seogizxy1Dmods[:,0:3],seogizxy1Dmods[:,3],mesh3d.gridCC,'linear')
mod3d_zxy1dMod[airInd] = np.log(1e-8)
mod3d_zyx1dMod = griddata(seogizyx1Dmods[:,0:3],seogizyx1Dmods[:,3],mesh3d.gridCC,'linear')
mod3d_zyx1dMod[airInd] = np.log(1e-8)

In [31]:
simpeg.Utils.meshutils.writeVTRFile('model3D_zxy1Dinv_paddedMesh.vtr',mesh3d,{'S/m':np.exp(mod3d_zxy1dMod)})
simpeg.Utils.meshutils.writeVTRFile('model3D_zyx1Dinv_paddedMesh.vtr',mesh3d,{'S/m':np.exp(mod3d_zyx1dMod)})

In [68]:
%matplotlib qt
m_0 = np.log(problem.sigmaPrimary[problem.mapping.sigmaMap.maps[-1].indActive])
simpegmt.Utils.dataUtils.plotMT1DModelData(problem,[m_0,mopt])
plt.suptitle('Target misfit-smooth False')
plt.show()


In [ ]:
np.exp(m_0)

In [ ]:
import multiprocessing, os, numpy as np
from glob import glob
import subprocess

def runMT3Dfwd(path):
    """ Worker function """
    orgDir = os.getcwd()
    print 'Starting in {:s}\n'.format(orgDir)
    
    os.chdir(path)
    print '########### \nRunning in {:s}\n###########'.format(os.getcwd())
    cmd = '/tera_raid/gudni/Codes/ZTEM_MT3Dinv_06082014/MT3Dfwd'
    # os.system(cmd)
    subprocess.call(cmd)
    os.chdir(orgDir)


if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=12)
    foldList = glob('MT3Dfwd/model100w*/Freq*')
    
    pool.map(runMT3Dfwd,foldList)
    pool.cself.mesh.getEdgeInnerProductlose()
    pool.join()

In [ ]:
locs=rec2ndarr(uniLocs)

In [ ]:
%debug

In [ ]:
simpeg.mkvc(locs[0,:],2).T

In [ ]:
simpegmt.DataMT.DataMT

In [ ]:
%matplotlib qt
sys.path.append('/home/gudni/Dropbox/code/python/MTview/')
import interactivePlotFunctions as iPf
dataRec = mt1DdataList[0].toRecArray('Complex')
iPf.MTinteractiveMap([dataRec])

In [ ]:
simpegmt.Utils.dataUtils.getAppRes(mt1DdataList[0])